In [1]:
from nna.exp.megan import knapsackutils

In [2]:
solution_by_location_name = knapsackutils.main(version="split_by_distr")


OrderedDict([('Site ID', '12'), ('Anthro/Bio', 'Bio/Anth'), ('Category', 'Bird/Plane'), ('Specific Category', ''), ('Comments', 'raven, songbirds, long tailed ducks with aircraft in background'), ('', ''), ('File Name', 'S4A10268_20190610_103000_bio_anth.wav'), ('Could not decice', 'FALSE'), ('Songbird', 'FALSE'), ('Water Bird', 'FALSE'), ('Insect', 'FALSE'), ('Running Water', 'FALSE'), ('Rain', 'FALSE'), ('Cable', 'FALSE'), ('Wind', 'FALSE'), ('Vehicle', 'FALSE'), ('Aircraft', 'FALSE'), ('Date dd/mm/yy)', '6/6/2019'), ('Start Time', '1:05:22'), ('End Time', '1:05:53'), ('#VALUE!', '0:00:31')])


/home/enis/projects/nna/src/nna/exp/megan/knapsackutils.py:111: RuntimeWarning: invalid value encountered in true_divide
  _P = P / norm(P, ord=1)


1.0.0 : 0.68, 0.13, 0.19, 47
3.0.0 : 0.48, 0.26, 0.26, 46
X.X.X : 0.5, 0.25, 0.25, 24
1.1.10 : 0.63, 0.22, 0.15, 459
1.1.0 : 0.63, 0.14, 0.24, 837
1.3.0 : 0.52, 0.4, 0.087, 161
1.1.8 : 0.49, 0.19, 0.32, 88
1.1.7 : 0.38, 0.32, 0.3, 160
1.2.4 : 1.0, 0.0, 0.0, 2
0.2.0 : 0.78, 0.22, 0.0, 23
1.1.5 : 1.0, 0.0, 0.0, 3
0.0.0 : 0.93, 0.017, 0.052, 58
1.2.0 : 0.0, 0.0, 1.0, 1
1.1.3 : 0.94, 0.062, 0.0, 32
2.1.0 : 0.22, 0.56, 0.22, 18
0.1.0 : 1.0, 0.0, 0.0, 23
0.4.0 : 0.62, 0.38, 0.0, 8


In [4]:
solution_by_location_name

[['45',
  '38',
  '48',
  '39',
  '11',
  '44',
  '46',
  '17',
  '20',
  '50',
  '13',
  '25',
  '21',
  '29',
  '19',
  '16',
  '24',
  '37'],
 ['18', '31', '34', '27', '32', '33', '47', '41', '22', '15'],
 ['30', '12', '14', '36', '40', '49']]

In [2]:

import numpy as np

from pathlib import Path
from collections import Counter

import runconfigs # type: ignore
import modelarchs # type: ignore

import nna.exp.megan as megan
from nna.exp import runutils

/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
# OVER WRITE CONFIG
runconfigs.TAXO_COUNT_LIMIT = 0

In [4]:


def prepare_dataset():

    taxo_count_limit = runconfigs.TAXO_COUNT_LIMIT
    sample_length_limit = runconfigs.SAMPLE_LENGTH_LIMIT
    taxonomy_file_path = runconfigs.TAXONOMY_FILE_PATH

    megan_labeled_files_info_path = runconfigs.MEGAN_LABELED_FILES_INFO_PATH

    csv4megan_excell_clenaed = runconfigs.CSV4MEGAN_EXCELL_CLEANED

    ignore_files = runconfigs.IGNORE_FILES

    excerpt_length = runconfigs.EXCERPT_LENGTH
    excell_names2code = runconfigs.EXCELL_NAMES2CODE
    dataset_name_v = runconfigs.DATASET_NAME_V

    audio_dataset, deleted_files = megan.preparedataset.run( # type: ignore
        megan_labeled_files_info_path,
        taxonomy_file_path,
        csv4megan_excell_clenaed,
        ignore_files,
        excerpt_length,
        sample_length_limit,
        taxo_count_limit,
        excell_names2code=excell_names2code,
        dataset_name_v=dataset_name_v)

    audio_dataset.load_audio_files(runconfigs.AUDIO_DATA_CACHE_PATH)
    audio_dataset.pick_channel_by_clipping()

    return audio_dataset, deleted_files

In [5]:
audio_dataset, _ = prepare_dataset()

/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/meganLabeledFiles_wlenV1.txt
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite
4 files are missing corresponding to excell entries
'-> 5 number of samples are DELETED due to ignore_files and missing_audio_files'
-> 415 samples DELETED because they are not in the excell

-> 0 samples DELETED because they do not have the taxo info coming from excell

-> classes that do not have enough data:
[REMOVED!]

-> classes that have enough data:
['other-biophony'] 56.0
['other-insect'] 140.0
['other-bird'] 661.0
['songbirds'] 392.0
['duck-goose-swan'] 183.0
['grouse-ptarmigan'] 59.0
['other-anthrophony'] 66.0
['other-mammal'] 0.0
['other-silence'] 20.0
['unknown-sound'] 2.0
['other-aircraft'] 107.0
['seabirds'] 1.0
['canids'] 1.0
['loons'] 29.0
['other-car'] 37.0
['other-flare'] 11.0
['other-rain'] 20.0
('-> 0 number of samples are deleted because their taxonomy category does not '
 'have enough data.')
-> classes that do not have e

In [6]:
from run import dataset_generate_samples
from run import data_to_samples


In [7]:
audio_dataset = dataset_generate_samples(audio_dataset,
                                             runconfigs.EXCERPT_LENGTH)

In [8]:
def taxo_counts(audio_dataset,prnt=False):
    taxo_dist=Counter()
    taxo2loc_dict = {}

    for sound_ins in audio_dataset.values():
        taxo_dist+=Counter({sound_ins.taxo_code:len(sound_ins.samples)})
        
        taxo2loc_dict.setdefault(sound_ins.taxo_code, Counter({}))
        taxo2loc_dict[taxonomy_code] = taxo2loc_dict[
                taxonomy_code] + Counter({site_id: 1})
        
    counted_items=list(taxo_dist.items())
    counted_items=sorted(counted_items)
    counted_items = [(i[0].split('.'),i[1]) for i in counted_items]
    if prnt:
        for i in counted_items:
            print(audio_dataset.taxonomy[i[0]][0],i[1])

    return counted_items

def taxo2loc_counts(audio_dataset,prnt=False):
    taxo2loc_dict = {}

    for sound_ins in audio_dataset.values():
        taxo2loc_dict.setdefault(sound_ins.taxo_code, Counter({}))
        taxo2loc_dict[sound_ins.taxo_code] = taxo2loc_dict[
                sound_ins.taxo_code] + Counter({sound_ins.location_id: len(sound_ins.samples)})
        

    return taxo2loc_dict

In [9]:
taxo2loc_dict=taxo2loc_counts(audio_dataset)

In [10]:
# taxo2loc_dict 

In [11]:
from nna.exp.megan.knapsackutils import *

In [33]:
from tabulate import tabulate

In [12]:
# taxo2loc_dict

In [13]:
# taxo2loc_dict

In [15]:
# taxo_size2= {i:0 for i in taxo2loc_dict2.keys()}
# for taxo,counts in taxo2loc_dict2.items():
#     for c in counts.values():
#         taxo_size2[taxo]+=c

In [16]:
taxo_size= {i:0 for i in taxo2loc_dict.keys()}
for taxo,counts in taxo2loc_dict.items():
    for c in counts.values():
        taxo_size[taxo]+=c

In [18]:
# # difference between number of files and number of samples (10 seconds)
# sorted(taxo_size.items())
# for m in taxo_size:
#     print(taxo_size[m],taxo_size2[m])

In [19]:
def find_upper_taxo(taxo):

    if '.' in taxo:
        taxo_a = taxo.split('.')
    else:
        taxo_a = taxo[:]

    if set(taxo_a)==set('X'):
        return '.'.join(taxo_a)
    if 'X' in taxo_a:
        taxo_a = [x if x!='X' else '0' for x in taxo_a]
    # -1 because we do not change first bit
    for i in range(len(taxo_a)-1):
        if taxo_a[-(i+1)]=='0':
            continue
        else:
            taxo_a[-(i+1)] = '0'
            break
    
    taxo_a='.'.join(taxo_a)
    return taxo_a

input_exptected_input =[('1.3.0','1.0.0'),
('1.1.0','1.0.0'),
('1.1.10','1.1.0'),
('1.1.0','1.0.0'),
('1.1.7','1.1.0'),
('1.1.0','1.0.0'),
('1.1.8','1.1.0'),
('1.1.0','1.0.0'),
('1.2.0','1.0.0'),
('0.2.0','0.0.0'),
('1.1.5','1.1.0'),
('1.1.0','1.0.0'),
('1.2.4','1.2.0'),
('1.2.0','1.0.0'),
('1.1.3','1.1.0'),
('1.1.0','1.0.0'),
('0.1.0','0.0.0'),
('0.4.0','0.0.0'),
('2.1.0','2.0.0'),
('1.X.X','1.0.0'),             
]
def test_upper_taxo(a_input,expected_input):
    assert expected_input==find_upper_taxo(a_input)

for a_input,expected_input in input_exptected_input:
    test_upper_taxo(a_input,expected_input)
# test_upper_taxo()

taxo2loc_dict_merged={}
# size_limit=31
for org_taxo,size in taxo_size.items():
#     if size>size_limit:
    if taxo2loc_dict_merged.get(taxo,None) is not None:
        print('over writing, ERROR')
        print(taxo)
    taxo2loc_dict_merged[org_taxo]=taxo2loc_dict[org_taxo]
    upper_taxo = find_upper_taxo(org_taxo)
    previous_taxo = org_taxo
    while upper_taxo!=previous_taxo:
        taxo2loc_dict_merged[upper_taxo]=taxo2loc_dict_merged.get(upper_taxo,Counter())+taxo2loc_dict[org_taxo]
        previous_taxo = upper_taxo
        upper_taxo = find_upper_taxo(previous_taxo)
        
# upper_taxo('1.X.X')

In [20]:
# taxo2loc_dict_merged

In [21]:
# taxo2loc_dict_merged

In [22]:
# taxo2loc_dict2 = add_taxo_code2dataset(reader)
taxo2loc_dict_val = taxo2loc_dict.values()
#merge taxo counters
a=list(taxo2loc_dict_val)

all_taxo=Counter()
for m in a:
    all_taxo+=m


In [23]:
all_taxo

Counter({'11': 125,
         '12': 143,
         '14': 90,
         '17': 98,
         '18': 34,
         '27': 114,
         '30': 127,
         '31': 108,
         '34': 12,
         '39': 100,
         '44': 129,
         '45': 9,
         '46': 115,
         '48': 120,
         '50': 138,
         '19': 75,
         '20': 99,
         '21': 86,
         '24': 165,
         '32': 17,
         '38': 124,
         '40': 62,
         '41': 137,
         '13': 26,
         '16': 99,
         '22': 38,
         '25': 91,
         '29': 144,
         '36': 20,
         '37': 110,
         '47': 35,
         '49': 127,
         '33': 62,
         '15': 104})

In [24]:

def calculate_item_distr_distance(bins, target_distr):
    '''calculate largest distance between bins and target_distr

        Args:
            bins: list of lists, [[item1_count,...],[item1_count,...],[item1_count,...]..]
                 inner list contains count of each item in the bin. Bins has to 
                 have same length with target_distr and each index should 
                 correspond to same item.

    '''

    for a_bin in bins:
        assert len(a_bin) == len(target_distr)

    all_costs = []
    for a_bin in bins:
        cost = JSD(a_bin, target_distr)
        all_costs.append(cost)
    
    all_costs=[0.47 if np.isnan(x) else x for x in all_costs]
#     print('len all_costs',len(all_costs))
#     print('len cost ',all_costs)
    return np.max(all_costs), all_costs

In [25]:
np.mean([1,2,3,4])

2.5

In [26]:
def JSD(P, Q):
    '''

    from https://github.com/BirdVox/cramer2020icassp/blob/master/00_create_splits.py
    '''
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (kl_div(_P, _M) + kl_div(_Q, _M))



In [27]:
# bins=[[8,8,8],[2,2,2],[0,0,0]]
# target_distr=[3,3,3]
# bins=np.array(bins)
# target_distr=np.array(target_distr)

# a,b=calculate_item_distr_distance(bins, target_distr)
# a,b

In [28]:

def pick_solution_by_item_distr(item_weights, solutions, location2taxo_dict,
                                target_dist):
    '''Pick best solution has closest item dist between bins.
    
        target_dist = dict, keys are taxo and values are total counts in dataset
    
    '''
    best_sol = None
    lowest_cost = float('Inf')
    target_array = np.array(list(target_dist.values()))
    target_array = target_array/np.sum(target_array)

    for sol in solutions:
        # returns 3 bins, each one is a Counter-> taxo:total_count
        bins_as_taxo_dist = map_weight_sol2taxo_dist_sol(
            item_weights, sol, location2taxo_dict)
        # turn Counters into array with same order of target_dist
        bins_as_taxo_array = []
        for a_bin in bins_as_taxo_dist:
            bin_array = [a_bin.get(k, 0) for k in target_dist.keys()]
            bins_as_taxo_array.append(bin_array)
        
        cost, xx = calculate_item_distr_distance(bins_as_taxo_array,
                                                target_array)
    
#         print('-------')
#         print(bins_as_taxo_array)
#         print('xx')
#         print(xx)

        if lowest_cost > cost:
            best_sol = sol
            lowest_cost = cost

    return best_sol, lowest_cost


In [29]:
all_taxo

Counter({'11': 125,
         '12': 143,
         '14': 90,
         '17': 98,
         '18': 34,
         '27': 114,
         '30': 127,
         '31': 108,
         '34': 12,
         '39': 100,
         '44': 129,
         '45': 9,
         '46': 115,
         '48': 120,
         '50': 138,
         '19': 75,
         '20': 99,
         '21': 86,
         '24': 165,
         '32': 17,
         '38': 124,
         '40': 62,
         '41': 137,
         '13': 26,
         '16': 99,
         '22': 38,
         '25': 91,
         '29': 144,
         '36': 20,
         '37': 110,
         '47': 35,
         '49': 127,
         '33': 62,
         '15': 104})

In [30]:
def run(taxo2loc_dict,audio_dataset,all_taxo):
    # weights and values for knapsack
    # counts of taxonomies are weights
    # bins are train,validation,test
    weights = list(all_taxo.values())
    values = weights

    #solve knapsack possible distributions around ~(60,20,20)
    bin_capacities_w_solutions = solve_knapsack4combinations(weights,values)
    
    # remove bin_capacity info from solutions
    solutions = [solution for bin_capacity,solution in bin_capacities_w_solutions]

    # switch dict keys with key of value dicts
    item_weights=all_taxo
    location2taxo_dict = reverse_taxo2loc_dict(taxo2loc_dict)

    dataset_taxo_distr = {k:sum(v.values()) for k,v in  taxo2loc_dict.items()}

    target_dist = dataset_taxo_distr

    best_sol, lowest_cost = pick_solution_by_item_distr(item_weights, solutions, location2taxo_dict, target_dist)

    solution_by_location_name = map_weights2indexes(item_weights, best_sol)

    # print solutions
    solution_by_taxo_dist=[]
    for sol_bin in solution_by_location_name:
        taxo_counter = [location2taxo_dict[loc] for loc in sol_bin]
        tc=Counter()
        for c in taxo_counter:
            tc+=c
        solution_by_taxo_dist.append(tc)


    t_dist = {}
    table=[]
    for taxo in taxo2loc_dict.keys():
        train=solution_by_taxo_dist[0][taxo]
        val=solution_by_taxo_dist[1][taxo]
        test = solution_by_taxo_dist[2][taxo]
        total = (train+val+test)
        t_dist[taxo]=(train/total,val/total,test/total)
        taxo_name = audio_dataset.taxonomy[taxo][0].replace('other-','')
        table.append([taxo_name,f'{train/total:1.2}', f'{val/total:1.2}', f'{test/total:1.2}', f'{total}'])
        print(f"{taxo_name} : {train/total:1.2}, {val/total:1.2}, {test/total:1.2}, {total}")
    print(tabulate(table, headers=['name','train','val','test','total']))  
    return solution_by_location_name,solution_by_taxo_dist

In [65]:
taxo='1.3.0'
train=solution_by_taxo_dist[0][taxo]
val=solution_by_taxo_dist[1][taxo]
test = solution_by_taxo_dist[2][taxo]
total = (train+val+test)
print(train,val,test,total)

222 25 6 253


0.09881422924901186

In [34]:
solution_by_location_name,solution_by_taxo_dist=run(taxo2loc_dict_merged,audio_dataset,all_taxo)

biophony : 0.64, 0.18, 0.18, 2691
insect : 0.88, 0.099, 0.024, 253
bird : 0.62, 0.18, 0.2, 2351
songbirds : 0.73, 0.15, 0.12, 674
duck-goose-swan : 0.56, 0.17, 0.27, 284
grouse-ptarmigan : 0.73, 0.23, 0.047, 128
anthrophony : 0.57, 0.23, 0.2, 284
mammal : 1.0, 0.0, 0.0, 3
silence : 0.32, 0.42, 0.26, 53
unknown-sound : 0.42, 0.25, 0.33, 24
aircraft : 0.37, 0.45, 0.18, 120
seabirds : 1.0, 0.0, 0.0, 3
canids : 1.0, 0.0, 0.0, 2
loons : 0.038, 0.65, 0.31, 52
car : 0.84, 0.068, 0.091, 44
flare : 0.59, 0.41, 0.0, 17
rain : 0.0, 0.61, 0.39, 31
geology : 0.0, 0.61, 0.39, 31
name                train    val    test    total
----------------  -------  -----  ------  -------
biophony            0.64   0.18    0.18      2691
insect              0.88   0.099   0.024      253
bird                0.62   0.18    0.2       2351
songbirds           0.73   0.15    0.12       674
duck-goose-swan     0.56   0.17    0.27       284
grouse-ptarmigan    0.73   0.23    0.047      128
anthrophony         0.57   0

/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


In [ ]:
table=[[] for set in range(3+1)]


for i,loc in enumerate(location_ids):
    table[i+1].append(loc)
    for taxo in taxo_codes:
        table[i+1].append(table_dict[loc][taxo])
table=sorted(table)

table[0].append('total')
for taxo in taxo_codes:
    table[0].append(total[taxo])
    
taxo_codes_names =[audio_dataset.taxonomy[i][0].replace('other-','')[:6] for i in taxo_codes] 
print(tabulate(table, headers=['loc']+taxo_codes_names))

In [201]:
taxo2loc_dict_merged['1.1.8']

Counter({'11': 6,
         '15': 5,
         '16': 17,
         '20': 15,
         '21': 2,
         '25': 1,
         '31': 36,
         '37': 2,
         '38': 5,
         '40': 28,
         '49': 11})

In [189]:
taxo2loc_dict_merged['0.4.0']

Counter({'25': 7, '27': 10})

In [51]:
t_dist = {}
table=[]
for taxo in taxo2loc_dict.keys():
    train=solution_by_taxo_dist[0][taxo]
    val=solution_by_taxo_dist[1][taxo]
    test = solution_by_taxo_dist[2][taxo]
    total = (train+val+test)
    t_dist[taxo]=(train/total,val/total,test/total)
    taxo_name = audio_dataset.taxonomy[taxo][0].replace('other-','')
    table.append([taxo_name,f'{train/total:1.2}', f'{val/total:1.2}', f'{test/total:1.2}', total])
    print(f"{taxo_name} : {train/total:1.2}, {val/total:1.2}, {test/total:1.2}, {total}")
    
table.sort(key=lambda x:x[-1],reverse=True)
print(tabulate(table, headers=['name','train','val','test','total']))

biophony : 0.64, 0.18, 0.18, 2691
insect : 0.88, 0.099, 0.024, 253
bird : 0.62, 0.18, 0.2, 2351
songbirds : 0.73, 0.15, 0.12, 674
duck-goose-swan : 0.56, 0.17, 0.27, 284
grouse-ptarmigan : 0.73, 0.23, 0.047, 128
anthrophony : 0.57, 0.23, 0.2, 284
mammal : 1.0, 0.0, 0.0, 3
silence : 0.32, 0.42, 0.26, 53
unknown-sound : 0.42, 0.25, 0.33, 24
aircraft : 0.37, 0.45, 0.18, 120
seabirds : 1.0, 0.0, 0.0, 3
canids : 1.0, 0.0, 0.0, 2
loons : 0.038, 0.65, 0.31, 52
car : 0.84, 0.068, 0.091, 44
flare : 0.59, 0.41, 0.0, 17
rain : 0.0, 0.61, 0.39, 31
name                train    val    test    total
----------------  -------  -----  ------  -------
biophony            0.64   0.18    0.18      2691
bird                0.62   0.18    0.2       2351
songbirds           0.73   0.15    0.12       674
duck-goose-swan     0.56   0.17    0.27       284
anthrophony         0.57   0.23    0.2        284
insect              0.88   0.099   0.024      253
grouse-ptarmigan    0.73   0.23    0.047      128
aircraft

In [38]:
table[0]

['aircraft', '0.37', '0.45', '0.18', '120']

In [67]:
solution_by_location_name

[['12',
  '14',
  '27',
  '49',
  '31',
  '39',
  '44',
  '45',
  '48',
  '19',
  '16',
  '21',
  '38',
  '41',
  '20',
  '29',
  '37',
  '15'],
 ['17', '46', '50', '32', '33', '25', '40'],
 ['11', '18', '34', '24', '13', '22', '36', '47', '30']]

In [238]:
print(tabulate(table, headers=['name','train','val','test','total']))

name                train    val    test    total
----------------  -------  -----  ------  -------
biophony             0.63  0.17    0.2         84
insect               0.78  0.11    0.11       337
bird                 0.73  0.14    0.12      1294
songbirds            0.75  0.096   0.15       674
duck-goose-swan      0.75  0.13    0.12      1578
grouse-ptarmigan     0.75  0.13    0.11      1422
anthrophony          0.64  0.049   0.31       103
silence              0.75  0.15    0.094       53
unknown-sound        0.88  0.042   0.083       24
aircraft             0.45  0.3     0.24       223
loons                0.73  0.14    0.13      1346
car                  0.72  0.034   0.24       147
rain                 0.32  0.45    0.23        31
